As the generation details of the **PhysOrg** descriptors is clearly illustrated in the **Supplementary Inforamtion**, this notebook only demonstrate how to generate the other descritors (**OneHot**, **RDKit**, **MF**, **MBTR**, **LMBTR**) used in the benchmark procedure.

# Load dependecies

In [12]:
import pandas as pd
import numpy as np
import glob
from rdkit import Chem
from rdkit.Chem import AllChem,Descriptors
from ase.io import read as ASE_read
from utils.DescriptorProcess import get_atom_species,maxminscale,process_desc,genOneHot,\
getmorganfp,calc_rdkit_desc,calc_Dscribe_Desc

descs = [desc_name[0] for desc_name in Descriptors._descList]

# Data Process

In [2]:
dataset = pd.read_csv('./datasets/dataset.csv',index_col=0)
biaryl_smiles = dataset['Biaryl'].to_numpy()
olefin_smiles = dataset['Olefin'].to_numpy()
tdg_smiles = dataset['TDG'].to_numpy()
additive_smiles = dataset['Additive'].to_numpy()  ## namely electrolyte
additive_smiles = np.array([item if isinstance(item,str) else "" for item in additive_smiles])
solvent_smiles = dataset['Solvent'].to_numpy()
current = dataset['Current(mA)'].to_numpy().reshape(-1,1)
temp = dataset['Temperature'].to_numpy().reshape(-1,1)
ee = dataset['ee'].to_numpy()
ddG = dataset['ddG(kcal/mol)'].to_numpy()

biaryl_smi_set = sorted(list(set(biaryl_smiles)))
olefin_smi_set = sorted(list(set(olefin_smiles)))
tdg_smi_set = sorted(list(set(tdg_smiles)))
add_smi_set = sorted(list(set(additive_smiles)))
sol_smi_set = sorted(list(set(solvent_smiles)))
dataset

,Biaryl,mole of Biaryl/mmol,Olefin,mole of Olefin/mmol,Product,Catalyst,Catalyst Concentration/%,TDG,TDG Concentration/%,Additive,...,Solvent Ratio,Volume of Solvent/ml,Reaction Time/h,Anode,Cathode,Current(mA),Temperature,Yield,ee,ddG(kcal/mol)
0,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F,...,NaN,4.5,14,grahite felt,Pt,1.5,60,0.57,0.99,3.507522
1,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F,...,NaN,4.5,14,grahite felt,Pt,0.0,60,0.20,0.99,3.507522
2,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],5,CC(C)(C)[C@H](N)C(=O)O,10,CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F,...,NaN,4.5,14,grahite felt,Pt,1.5,60,0.38,0.99,3.507522
3,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],5,CC(C)(C)[C@H](N)C(=O)O,10,CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F,...,NaN,4.5,14,grahite felt,Pt,4.0,60,0.33,0.99,3.507522
4,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F,...,NaN,4.5,14,grahite felt,Pt,2.0,60,0.41,0.99,3.507522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=Cc1ccccc1,0.6,O=Cc1ccccc1-c1c(/C=C/c2ccccc2)ccc2ccccc12,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,[Li]OC(C)=O,...,NaN,4.5,16,grahite felt,Pt,1.5,60,0.19,0.86,1.714028
123,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)Oc1ccccc1-c1ccccc1,0.6,O=Cc1ccccc1-c1c(/C=C/C(=O)Oc2ccccc2-c2ccccc2)c...,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,[Li]OC(C)=O,...,NaN,4.5,20,grahite felt,Pt,1.0,60,0.55,0.97,2.772852
124,O=Cc1ccccc1-c1cccc2ccccc12,0.2,C=CC(=O)OCC,0.6,CCOC(=O)/C=C/c1ccc2ccccc2c1-c1ccccc1C=O,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,[Li]OC(C)=O,...,NaN,4.5,16,grahite felt,Pt,1.5,60,0.55,0.96,2.578852
125,O=Cc1ccc2ccccc2c1-c1ccccc1,0.2,C=CC(=O)OCCCC,0.6,CCCCOC(=O)/C=C/c1ccccc1-c1c(C=O)ccc2ccccc12,CC(=O)[O-].CC(=O)[O-].[Pd+2],10,CC(C)(C)[C@H](N)C(=O)O,20,[Li]OC(C)=O,...,NaN,4.5,20,grahite felt,Pt,1.5,60,0.49,0.97,2.772852


# Generate Condition Features

Different encoding approaches only applied to biaryls, olefins and TDGs. The same physical features are used to encode solvents and additives (namely electrolytes).

In [3]:
sol_set = sorted(list(set(solvent_smiles)))
add_set = sorted(list(set(additive_smiles)))

sol_desc_df = pd.read_csv('./descriptors/SolPhysOrg.csv')
add_desc_df = pd.read_csv('./descriptors/AddPhysOrg.csv')
sol_desc_name = list(sol_desc_df.columns)[1:]
sol_desc_name = np.array(['Sol '+n+' 1' for n in sol_desc_name] + ['Sol '+n+' 2' for n in sol_desc_name])
sol_desc = sol_desc_df.to_numpy()[:,1:]
sol_smiles = sol_desc_df['SMILES'].to_list()
sol_desc_map_ = {smi:desc for smi,desc in zip(sol_smiles,sol_desc)}
sol_desc_map = {}
for smi in sol_set:
    smi_split = smi.split('.')
    smi_split.remove('CC(=O)O')
    if smi_split == []:
        sol_desc_map[smi] = np.concatenate([sol_desc_map_['CC(=O)O'],np.zeros(len(sol_desc_map_['CC(=O)O']))])
    else:
        sol_desc_map[smi] = np.concatenate([sol_desc_map_['CC(=O)O'],sol_desc_map_[smi_split[0]]])

add_desc_name = np.array(list(add_desc_df.columns)[1:])
add_desc_name = np.array(['Add '+n for n in add_desc_name])
add_desc = add_desc_df.to_numpy()[:,1:]
add_smiles = [item if isinstance(item,str) else '' for item in add_desc_df['SMILES'].to_list()]
add_desc_map = {smi:desc for smi,desc in zip(add_smiles,add_desc)}

condition_desc = maxminscale(process_desc(np.array([np.concatenate([add_desc_map[additive_smiles[i]],
                             sol_desc_map[solvent_smiles[i]],np.array([float(current[i]),temp[i][0]])])\
                             for i in range(len(dataset))])))

# Generate Different Descriptors for Biaryl, Olefin and TDG

## Generate Geometry Map

Load xTB-optimized geometry *.xyz* files to generate geometry map, which is used to generate 3D descriptors (eg. MBTR, LMBTR).

In [4]:
all_smi_set = biaryl_smi_set + olefin_smi_set + tdg_smi_set
xtbopt_files = glob.glob('./Coordinates/naive/*.xyz')

smi_atoms_map = {}
smi_mol_map = {}
for file in xtbopt_files:
    with open(file,'r') as fr:
        lines = fr.readlines()
    smi = lines[1].strip()
    
    
    mol = AllChem.AddHs(Chem.MolFromSmiles(smi))
    atoms = ASE_read(file,format='xyz')
    smi_atoms_map[smi] = atoms
    smi_mol_map[smi] = mol

In [5]:
reactant_atom_species = get_atom_species(biaryl_smi_set+olefin_smi_set,smi_mol_map)
tdg_atom_species = get_atom_species(tdg_smi_set,smi_mol_map)

## One-Hot

**Note:** For demonstration, only compounds in the collected 127 data are used to generate all descriptors. The compounds in external test set should also be encoded in the same way for making prediction on the external test set.

In [6]:
biaryl_onehot_map = genOneHot(biaryl_smi_set)
olefin_onehot_map = genOneHot(olefin_smi_set)
tdg_onehot_map = genOneHot(tdg_smi_set)
smi_onehot_map = {}
smi_onehot_map.update(biaryl_onehot_map)
smi_onehot_map.update(olefin_onehot_map)
smi_onehot_map.update(tdg_onehot_map)

In [7]:
pd.DataFrame.from_dict(biaryl_onehot_map).T

,0,1,2,3,4,5,6,7,8,9,10,11
CCc1ccccc1-c1cc(F)ccc1C=O,1,0,0,0,0,0,0,0,0,0,0,0
COc1cc(C=O)c(-c2cccc3ccccc23)cc1OC,0,1,0,0,0,0,0,0,0,0,0,0
Cc1ccccc1-c1c(C=O)ccc2ccc3ccccc3c12,0,0,1,0,0,0,0,0,0,0,0,0
Cc1ccccc1-c1cc(F)ccc1C=O,0,0,0,1,0,0,0,0,0,0,0,0
Cc1ccccc1-c1ccccc1C=O,0,0,0,0,1,0,0,0,0,0,0,0
O=Cc1cc(C(F)(F)F)ccc1-c1cccc2ccccc12,0,0,0,0,0,1,0,0,0,0,0,0
O=Cc1cc2c(cc1-c1cccc3ccccc13)OCO2,0,0,0,0,0,0,1,0,0,0,0,0
O=Cc1ccc2ccc3ccccc3c2c1-c1cccc2ccccc12,0,0,0,0,0,0,0,1,0,0,0,0
O=Cc1ccc2ccccc2c1-c1cccc2ccccc12,0,0,0,0,0,0,0,0,1,0,0,0
O=Cc1ccc2ccccc2c1-c1ccccc1,0,0,0,0,0,0,0,0,0,1,0,0


In [8]:
pd.DataFrame.from_dict(olefin_onehot_map).T

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
C=CC(=O)NC(C)C,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)OC,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)OCC,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)OCCCC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)OCc1ccccc1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)Oc1ccc(Br)cc1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)Oc1ccc(C(C)=O)cc1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)Oc1ccc(F)cc1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)Oc1ccc([N+](=O)[O-])cc1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
C=CC(=O)Oc1cccc(OC)c1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
pd.DataFrame.from_dict(tdg_onehot_map).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
CC(C)(C)OC(=O)C[C@H](N)C(=O)O,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H](N)C(=O)N[C@H](C(=O)O)C(C)(C)C,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H](N)C(=O)O,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H](NC(=O)C[NH3+])C(=O)O.[Cl-],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])Cc1ccccc1)C(=O)O.[Cl-],0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])c1ccccc1)C(=O)O.[Cl-],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)(C)[C@H]([NH3+])C(=O)N[C@H](C(=O)O)C(C)(C)C.[Cl-],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)[C@H](N)C(=O)O,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
CC(C)[C@H](NC(=O)[C@@H]([NH3+])C(C)(C)C)C(=O)O.[Cl-],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
CC(O)[C@H](N)C(=O)O,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


## RDKit/FP

In [10]:
smi_rdkit_map = {smi:calc_rdkit_desc(smi_mol_map[smi]) for smi in all_smi_set}
smi_mf_map = {smi:getmorganfp(smi_mol_map[smi]) for smi in all_smi_set}

In [13]:
rdkit_desc_df = pd.DataFrame.from_dict(smi_rdkit_map).T
rdkit_desc_df.columns = descs
rdkit_desc_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
CCc1ccccc1-c1cc(F)ccc1C=O,14.391043,-3.481147,14.391043,0.995592,0.728727,228.266,215.162,228.095043,86.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COc1cc(C=O)c(-c2cccc3ccccc23)cc1OC,12.418245,-3.361375,12.418245,0.581094,0.671108,292.334,276.206,292.109944,110.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1c(C=O)ccc2ccc3ccccc3c12,12.622036,-3.253489,12.622036,0.530991,0.338259,296.369,280.241,296.120115,110.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1cc(F)ccc1C=O,14.221338,-3.140075,14.221338,0.866667,0.697713,214.239,203.151,214.079393,80.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1ccccc1C=O,11.771312,-3.078851,11.771312,0.736921,0.671952,196.249,184.153,196.088815,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc(C(F)(F)F)ccc1-c1cccc2ccccc12,13.442683,-5.339704,13.442683,0.599316,0.583913,300.279,289.191,300.076200,110.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc2c(cc1-c1cccc3ccccc13)OCO2,12.223077,-2.795850,12.223077,0.445086,0.662359,276.291,264.195,276.078644,102.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccc3ccccc3c2c1-c1cccc2ccccc12,12.821676,-1.601173,12.821676,0.488086,0.261932,332.402,316.274,332.120115,122.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1cccc2ccccc12,12.451756,-1.548446,12.451756,0.476068,0.445298,282.342,268.230,282.104465,104.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1ccccc1,12.081835,-1.495720,12.081835,0.446471,0.601254,232.282,220.186,232.088815,86.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
pd.DataFrame.from_dict(smi_mf_map).T

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
CCc1ccccc1-c1cc(F)ccc1C=O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COc1cc(C=O)c(-c2cccc3ccccc23)cc1OC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cc1ccccc1-c1c(C=O)ccc2ccc3ccccc3c12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cc1ccccc1-c1cc(F)ccc1C=O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cc1ccccc1-c1ccccc1C=O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O=Cc1cc(C(F)(F)F)ccc1-c1cccc2ccccc12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O=Cc1cc2c(cc1-c1cccc3ccccc13)OCO2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O=Cc1ccc2ccc3ccccc3c2c1-c1cccc2ccccc12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O=Cc1ccc2ccccc2c1-c1cccc2ccccc12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O=Cc1ccc2ccccc2c1-c1ccccc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## MBTR

In [15]:
mbtr_parameter_dict = {'k1':{
                             "geometry": {"function": "atomic_number"},
                             "grid": {"min": 0, "max": 8, "n": 10, "sigma": 0.1}},
                       'k2':{
                            "geometry": {"function": "inverse_distance"},
                            "grid": {"min": 0, "max": 4, "n": 10, "sigma": 0.1},
                            "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3}},
                       'k3':{
                            "geometry": {"function": "cosine"},
                            "grid": {"min": -1, "max": 4, "n": 10, "sigma": 0.1},
                            "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3}},
                       'periodic':False,'normalization':'l2_each'
                      }


biaryl_smi_mbtr_map = {}
olefin_smi_mbtr_map = {}
tdg_smi_mbtr_map = {}

for smi in biaryl_smi_set:
    biaryl_smi_mbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                               reactant_atom_species,mbtr_parameter_dict,type_='MBTR')
for smi in olefin_smi_set:
    olefin_smi_mbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                               reactant_atom_species,mbtr_parameter_dict,type_='MBTR')
for smi in tdg_smi_set:
    tdg_smi_mbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                            tdg_atom_species,mbtr_parameter_dict,type_='MBTR')

In [16]:
pd.DataFrame.from_dict(biaryl_smi_mbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,3310,3311,3312,3313,3314,3315,3316,3317,3318,3319
CCc1ccccc1-c1cc(F)ccc1C=O,0.0,0.658825,0.000283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COc1cc(C=O)c(-c2cccc3ccccc23)cc1OC,0.0,0.644161,0.000276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1c(C=O)ccc2ccc3ccccc3c12,0.0,0.592650,0.000254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1cc(F)ccc1C=O,0.0,0.621674,0.000267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1ccccc1C=O,0.0,0.654572,0.000281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc(C(F)(F)F)ccc1-c1cccc2ccccc12,0.0,0.525677,0.000226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc2c(cc1-c1cccc3ccccc13)OCO2,0.0,0.554180,0.000238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccc3ccccc3c2c1-c1cccc2ccccc12,0.0,0.543601,0.000233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1cccc2ccccc12,0.0,0.559134,0.000240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1ccccc1,0.0,0.580890,0.000249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pd.DataFrame.from_dict(olefin_smi_mbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,3310,3311,3312,3313,3314,3315,3316,3317,3318,3319
C=CC(=O)NC(C)C,0.0,0.874843,0.000375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OC,0.0,0.804648,0.000345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCC,0.0,0.832358,0.000357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCCCC,0.0,0.857637,0.000368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCc1ccccc1,0.0,0.704479,0.000302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(Br)cc1,0.0,0.609338,0.000262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(C(C)=O)cc1,0.0,0.663764,0.000285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(F)cc1,0.0,0.609338,0.000262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc([N+](=O)[O-])cc1,0.0,0.581356,0.000250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1cccc(OC)c1,0.0,0.695891,0.000299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pd.DataFrame.from_dict(tdg_smi_mbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,940,941,942,943,944,945,946,947,948,949
CC(C)(C)OC(=O)C[C@H](N)C(=O)O,0.0,0.859732,0.000369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](N)C(=O)N[C@H](C(=O)O)C(C)(C)C,0.0,0.888648,0.000381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](N)C(=O)O,0.0,0.899013,0.000386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)C[NH3+])C(=O)O.[Cl-],0.0,0.890550,0.000382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])Cc1ccccc1)C(=O)O.[Cl-],0.0,0.833551,0.000358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])c1ccccc1)C(=O)O.[Cl-],0.0,0.826816,0.000355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H]([NH3+])C(=O)N[C@H](C(=O)O)C(C)(C)C.[Cl-],0.0,0.896066,0.000385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)[C@H](N)C(=O)O,0.0,0.897022,0.000385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)[C@H](NC(=O)[C@@H]([NH3+])C(C)(C)C)C(=O)O.[Cl-],0.0,0.895293,0.000384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(O)[C@H](N)C(=O)O,0.0,0.871668,0.000374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## LMBTR

In [19]:
lmbtr_parameter_dict = {"k2":{
                                "geometry": {"function": "distance"},
                                "grid": {"min": 0, "max": 5, "n": 100, "sigma": 0.1},
                                "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3},
                              },
                        "k3":{
                                "geometry": {"function": "cosine"},
                                "grid": {"min": -1, "max": 4, "n": 100, "sigma": 0.1},
                                "weighting": {"function": "exponential", "scale": 0.5, "cutoff": 1e-3},
                             },
                        "periodic":False,"normalization":"l2_each"
                       }

biaryl_smi_lmbtr_map = {}
olefin_smi_lmbtr_map = {}
tdg_smi_lmbtr_map = {}

for smi in biaryl_smi_set:
    biaryl_smi_lmbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                                reactant_atom_species,lmbtr_parameter_dict,type_='LMBTR')
    
for smi in olefin_smi_set:
    olefin_smi_lmbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                                reactant_atom_species,lmbtr_parameter_dict,type_='LMBTR')
    
for smi in tdg_smi_set:
    tdg_smi_lmbtr_map[smi] = calc_Dscribe_Desc(smi,smi_atoms_map,
                             tdg_atom_species,lmbtr_parameter_dict,type_='LMBTR')

In [20]:
pd.DataFrame.from_dict(biaryl_smi_lmbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,12590,12591,12592,12593,12594,12595,12596,12597,12598,12599
CCc1ccccc1-c1cc(F)ccc1C=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COc1cc(C=O)c(-c2cccc3ccccc23)cc1OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1c(C=O)ccc2ccc3ccccc3c12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1cc(F)ccc1C=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cc1ccccc1-c1ccccc1C=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc(C(F)(F)F)ccc1-c1cccc2ccccc12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1cc2c(cc1-c1cccc3ccccc13)OCO2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccc3ccccc3c2c1-c1cccc2ccccc12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1cccc2ccccc12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O=Cc1ccc2ccccc2c1-c1ccccc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
pd.DataFrame.from_dict(olefin_smi_lmbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,12590,12591,12592,12593,12594,12595,12596,12597,12598,12599
C=CC(=O)NC(C)C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCCCC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)OCc1ccccc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(Br)cc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(C(C)=O)cc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc(F)cc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1ccc([N+](=O)[O-])cc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C=CC(=O)Oc1cccc(OC)c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
pd.DataFrame.from_dict(tdg_smi_lmbtr_map).T

,0,1,2,3,4,5,6,7,8,9,...,5690,5691,5692,5693,5694,5695,5696,5697,5698,5699
CC(C)(C)OC(=O)C[C@H](N)C(=O)O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](N)C(=O)N[C@H](C(=O)O)C(C)(C)C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](N)C(=O)O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)C[NH3+])C(=O)O.[Cl-],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])Cc1ccccc1)C(=O)O.[Cl-],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H](NC(=O)[C@@H]([NH3+])c1ccccc1)C(=O)O.[Cl-],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)(C)[C@H]([NH3+])C(=O)N[C@H](C(=O)O)C(C)(C)C.[Cl-],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)[C@H](N)C(=O)O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(C)[C@H](NC(=O)[C@@H]([NH3+])C(C)(C)C)C(=O)O.[Cl-],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CC(O)[C@H](N)C(=O)O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
